In [15]:
from cdapython import Q,unique_terms,columns
import requests, json
from deepdiff import DeepDiff

In [16]:
postgres_url="https://cancerdata.ah-cda6.bee.envs-terra.bio/api/v1/boolean-query"
postgres_params={'dryRun':'false','includeCount':'false','offset':0,'limit':100}
headers={'accept':'application/json','Content-Type': 'application/json'}

In [17]:
def get_postgres_results(json_query):
    return requests.post(postgres_url, params=postgres_params, headers=headers, json=json_query)

In [25]:
myquery = Q('(stage = "IIA" OR stage = "IIB") AND primary_diagnosis_site = "Lung"').ORDER_BY("subject_id")

In [26]:
print(myquery.to_json())

{
    "node_type": "ORDERBY",
    "l": {
        "node_type": "ORDERBYVALUES",
        "value": "subject_id"
    },
    "r": {
        "node_type": "AND",
        "l": {
            "node_type": "OR",
            "l": {
                "node_type": "=",
                "l": {
                    "node_type": "column",
                    "value": "stage"
                },
                "r": {
                    "node_type": "quoted",
                    "value": "IIA"
                }
            },
            "r": {
                "node_type": "=",
                "l": {
                    "node_type": "column",
                    "value": "stage"
                },
                "r": {
                    "node_type": "quoted",
                    "value": "IIB"
                }
            }
        },
        "r": {
            "node_type": "=",
            "l": {
                "node_type": "column",
                "value": "primary_diagnosis_site"
            },
            "r": {
                "node_type": "quoted",
                "value": "Lung"
            }
        }
    }
}

In [27]:
bq_response = myquery.subject.run(verbose=False)._api_response
bq_sql = bq_response.query_sql
bq_results_json = bq_response.result

Total execution time: 0
                            min 4.289 sec 4289 ms

In [28]:
pg_response = get_postgres_results(myquery.to_dict())
pg_sql = pg_response.json()['query_sql']
pg_results_json = pg_response.json()['result']

Total execution time: 0
                            min 1.483 sec 1483 ms

In [29]:
diffs = DeepDiff(bq_results_json, pg_results_json,  ignore_order=True)

In [30]:
if (len(diffs) > 0):
    print(diffs)
    print(bq_sql)
    print(pg_sql)